In [11]:

using ITensors
using ITensors: position!
#using ITensorTDVP
using KrylovKit
using Printf
#using ProgressMeter
#ProgressMeter.ijulia_behavior(:append)

# Simple TDVP

Below is a code for a simple TDVP implementation (both for 1-site & 2-site variant). This is based on the ITensorTDVP. The function tdvp! takes the folllowing arguments.

**Initial wavefunction** <br>
**Hamiltonian :** (in MPO form) to time evolve the wave function<br>
**dt(time step)** <br>
**tf(final time )** : time upto which you want to evolve the system)<br>
**reverse_step_val :** Decides whether you need the right-to-left swap (Changes the accuracy O(dt^3) for true,O(dt^2) for no)

Time is entered as real for time evolution. The i is multiplied internally.

In [12]:
struct TDVP end
set_nsite!(PH::ProjMPO,nsite) = (PH.nsite = nsite)

function solver(H, t, psi0; kwargs...)
    solver_kwargs = (;
      ishermitian=get(kwargs, :ishermitian, true),
      issymmetric=get(kwargs, :issymmetric, true),
      tol=get(kwargs, :solver_tol, 1E-12),
      krylovdim=get(kwargs, :solver_krylovdim, 30),
      maxiter=get(kwargs, :solver_maxiter, 100),
      verbosity=get(kwargs, :solver_outputlevel, 0),
      eager=true,
    )
    psi, info = exponentiate(H, t, psi0; solver_kwargs...)
    return psi, info
end

function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end


function tdvp!(ψ::MPS,H::MPO,dt::Float64,tf::Float64,nsite::Int64,reverse_step_val; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    maxdim::Integer = get(kwargs, :maxdim, typemax(Int))
    mindim::Integer = get(kwargs, :mindim, 1)
    cutoff::Real = get(kwargs, :cutoff, 1E-16)
    
    #If progress bar is used
    #pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
   #Imaginary time step
    τ = 1im*dt
    
    N = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #Projected MPO
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    
    PH = ProjMPO(H)
    position!(PH,ψ,1)
    
    
    
    set_nsite!(PH, nsite)
    
    
    maxtruncerr = 0.0
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        current_time = dt*time_step
        for (b,ha) in sweepnext(N; ncenter=nsite)
                # Do 'forwards' evolution step
                
                
                set_nsite!(PH, nsite)
                #orthogonalise at position b
                
                position!(PH, ψ, b)
                if nsite == 1
                  #println(" Forward phi1 Selected")
                  phi1 = ψ[b]
                elseif nsite == 2
                  phi1 = ψ[b] * ψ[b + 1]
                end

                
                #orthogonalise at position b
                
                #phi1, info = exponentiate(PH, -τ/2, phi1; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                phi1, info = solver(PH, -τ/2, phi1)
                info.converged==0 && throw("exponentiate did not converge")
                
                spec = nothing
                
                
                if nsite == 1
                  ψ[b] = phi1
                  Sz = measure_Sz(ψ,b) 
                  #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
                  flush(stdout)
                  #println(" Forward phi1 Replaced")
                elseif nsite == 2
                  #Determine sweep direction
                  dir = ha == 1 ? "left" : "right"
                  spec = replacebond!(ψ,b,phi1;mindim,cutoff,normalize=normalize, ortho = dir, kwargs... )
                  maxtruncerr = max(maxtruncerr, spec.truncerr)
                end
                
                if(reverse_step_val == true)
                    if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))


                        b1 = (ha==1 ? b + 1 : b)
                        Δ = (ha==1 ? +1 : -1)

                        if nsite == 2
                            phi0 = ψ[b1]
                        elseif nsite == 1
                            #println(" SVD")
                            uinds = uniqueinds(phi1, ψ[b + Δ])
                            U, S, V = svd(phi1, uinds)
                            ψ[b] = U
                            phi0 = S * V
                            if ha == 1
                              ITensors.setleftlim!(ψ, b)
                            elseif ha == 2
                              ITensors.setrightlim!(ψ, b)
                            end
                        end

                        # evolve with single-site Hamiltonian backward in time.
                        # In the case of imaginary time-evolution this step
                        # is not necessary (see Ref. [1])




                        set_nsite!(PH, nsite - 1)
                        ITensors.position!(PH, ψ, b1)
                        phi0, info = exponentiate(PH, τ/2, phi0; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                        normalize && (phi0 ./= norm(phi0))
                        info.converged==0 && throw("exponentiate did not converge")


                        if nsite == 2
                            ψ[b1] = phi0
                            #print(" Reverse phi  calculated")
                        elseif nsite == 1
                            #println(" Reverse phi  calculated")
                            ψ[b + Δ] = phi0 * ψ[b + Δ]
                            if ha == 1
                              ITensors.setrightlim!(ψ, b + Δ + 1)
                            elseif ha == 2
                              ITensors.setleftlim!(ψ, b + Δ - 1)
                            end
                        end

                        set_nsite!(PH, nsite)



                    end
                    #if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))
                end
                #for if(reverse_step_val == true)
        end
        #for(b,ha)
            
            #Sz = measure_Sz(ψ,5) 
            #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
            flush(stdout)

        
        end
        #time elapsed
        
        
       
        
    end
    #External for loop for time_step
    
    ψ=ψ/norm(ψ)
end
#function


tdvp! (generic function with 1 method)

In [13]:
N = 5

s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
#ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ=randomMPS(s; linkdims=5)
#psi = productMPS(s, n -> isodd(n) ? "Up" : "Dn")


for j=1:N
    println("site : ",j," ",measure_Sz(ψ,j))
end

ψ4 = deepcopy(ψ)
ψ5 = deepcopy(ψ)

site : 1 0.2386134273978999
site : 2 -0.05119517564342503
site : 3 0.22737207953409858
site : 4 0.15806396061647587
site : 5 0.08394746188903343


MPS
[1] ((dim=2|id=610|"S=1/2,Site,n=1"), (dim=5|id=322|"Link,l=1"))
[2] ((dim=5|id=322|"Link,l=1"), (dim=2|id=526|"S=1/2,Site,n=2"), (dim=5|id=989|"Link,l=2"))
[3] ((dim=5|id=989|"Link,l=2"), (dim=2|id=941|"S=1/2,Site,n=3"), (dim=4|id=681|"Link,l=3"))
[4] ((dim=4|id=681|"Link,l=3"), (dim=2|id=3|"S=1/2,Site,n=4"), (dim=2|id=952|"Link,l=4"))
[5] ((dim=2|id=952|"Link,l=4"), (dim=2|id=564|"S=1/2,Site,n=5"))


In [14]:
ψ1 = deepcopy(ψ)
#2-site TDVP
tdvp!(ψ1,H,0.05,0.5,2,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.
for j=1:N
    println("site : ",j," ",measure_Sz(ψ1,j))
end



site : 1 0.21236707237566416
site : 2 -0.005802955189429249
site : 3 0.20524095594274205
site : 4 0.1712134493537829
site : 5 0.07378323131134688


In [15]:
ψ2 = deepcopy(ψ)

#1-site TDVP
tdvp!(ψ2,H,0.05,0.2,1,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ2,j))
end



site : 1 0.23426765026211294
site : 2 -0.043558234889115546
site : 3 0.22354079056896745
site : 4 0.1602923904594015
site : 5 0.08225915739271991


In [17]:
#TEBD
ψ3 = deepcopy(ψ)
cutoff = 1e-15

tau = 0.05
ttotal = 2.0


# Make gates (1,2),(2,3),(3,4),...
gates = ITensor[]
for j in 1:(N - 1)
s1 = s[j]
s2 = s[j + 1]
hj =
  op("Sz", s1) * op("Sz", s2) +
  1 / 2 * op("S+", s1) * op("S-", s2) +
  1 / 2 * op("S-", s1) * op("S+", s2)
Gj = exp(-im * tau / 2 * hj)
push!(gates, Gj)
end
# Include gates in reverse order too
# (N,N-1),(N-1,N-2),...
append!(gates, reverse(gates))
c=div(N,2)

for t in 0.0:tau:ttotal
    Sz = expect(ψ3, "Sz"; sites=c)
    #println("$t $Sz")

    t≈ttotal && break

    ψ3 = apply(gates, ψ3; cutoff)
    normalize!(ψ3)
end


#Measure magnetisation at each site
for j=1:N
    println("site : ",j," ",measure_Sz(ψ3,j))
end



site : 1 0.00534246424312276
site : 2 0.25388863757375435
site : 3 0.1657883707830905
site : 4 0.23146204681032553
site : 5 0.0003202343837784838


In [18]:
println(inner(ψ1,ψ2))

0.9697486538198146 + 0.05667986403869342im


In [19]:
println(inner(ψ1,ψ3))

0.36525005859403564 - 0.26072240902451255im


In [21]:
println(inner(ψ2,ψ3))

0.15512012945220924 - 0.29311228823270186im


In [29]:
println(ψ3)

MPS
[1] ((dim=2|id=165|"S=1/2,Site,n=1"), (dim=5|id=166|"Link,l=1"))
[2] ((dim=5|id=166|"Link,l=1"), (dim=2|id=920|"S=1/2,Site,n=2"), (dim=5|id=713|"Link,l=2"))
[3] ((dim=5|id=713|"Link,l=2"), (dim=2|id=580|"S=1/2,Site,n=3"), (dim=4|id=667|"Link,l=3"))
[4] ((dim=4|id=667|"Link,l=3"), (dim=2|id=984|"S=1/2,Site,n=4"), (dim=2|id=811|"Link,l=4"))
[5] ((dim=2|id=811|"Link,l=4"), (dim=2|id=59|"S=1/2,Site,n=5"))

